# Playlist Generation

This notebook combines the Emotion Recognition Model and the Sentence Embedding Model to generate playlists. After the user inputs text, the Emotion Recognition model calculates what the prevailing emotion is in the input text. Then, using the song emotion dictionary `emotion_dict.pkl`, all songs containing the same emotion are chosen. From these, the semantic embeddings are compared using cosine similarity, and the k most similar songs are returned in a playlist. 

In [ ]:
!pip install -U sentence-transformers

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from sentence_transformers import SentenceTransformer
from numpy import dot
from numpy.linalg import norm
from google.colab import drive
import pandas as pd
import numpy as np
import ast
import pickle
from random import shuffle, choice, sample
import tqdm
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

In [4]:
# List of all emotions to categorize by
emotions = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Import sentence embedding model
sts_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [7]:
# Import song dataset
data = pd.read_csv("drive/MyDrive/NLUColabFiles/spotify_millsongdata.csv")
data.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [8]:
# Remove newlines from text
lyrics = data.text
lyrics = [str(song).replace("\r", "").replace("\n", "") for song in lyrics]

In [10]:
# Import bert tokenizer
tokeniser = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [11]:
# Encode text 
def encode(corpus):
    encoded = tokeniser.encode_plus(corpus, max_length=128,
                                         add_special_tokens=True,
                                         return_attention_mask=True,
                                         truncation=True,
                                         return_tensors='pt',
                                         padding='max_length')

    return encoded['input_ids'], encoded['attention_mask']

In [12]:
# Import emotion classifier
classifier = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                          num_labels=6,
                                                          output_attentions=False,
                                                          output_hidden_states=False)

classifier.load_state_dict(torch.load('drive/MyDrive/NLUColabFiles/ERModel.zip'))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [13]:
# Method for classifying emotions in text
def make_classifier_prediction(input_text):
  ids, masks = encode(input_text)

  tensored = TensorDataset(ids, masks)
  sampler = SequentialSampler(tensored)
  dataloader = DataLoader(tensored, sampler=sampler)

  with torch.no_grad():
    for data in dataloader:
      id, mask = [x.to('cpu') for x in data]
      output = classifier(input_ids=id, attention_mask=mask)

      label = np.argmax(output.logits.cpu().detach().numpy(), axis=-1)
      
      return label[0]

In [15]:
# Open emotion dictionary for existing songs
with open('drive/MyDrive/NLUColabFiles/emotion_dict.pkl', 'rb') as f:
  emotion_dict = pickle.load(f)

emotion_dict['love'][0:20]

[1, 3, 4, 7, 10, 11, 17, 18, 19, 23, 24, 30, 39, 43, 46, 48, 50, 53, 55, 56]

In [16]:
# Method to calculate cosine similarities between input and a list of targets
def cosine_sim(inp, targets):
  cos_sim = []
  for target in targets:
    score = dot(inp,target)/(norm(inp)*norm(target))
    cos_sim.append(score)
  return cos_sim

In [17]:
# Method to get similar songs, given an input text
# Get the emotion of the input text, and find 5000 songs
# with that emotion. Then calculate the cosine similarity
# between the input text and matching songs,
# and return the k most similar songs (where k = playlist_length).
def get_songs_by_text(input_song, playlist_length):
  choices = []
  emotion_idx = make_classifier_prediction(input_song)
  emotion = emotions[emotion_idx]
  print(f"Detected Emotion: {emotion}")
  print("Running STS model")
  options = emotion_dict[emotion]
  options = sample(options, 5000)
  all_lyrics = []
  for option in options:
    song_lyrics = lyrics[option]
    all_lyrics.append(song_lyrics)
  
  embeddings = sts_model.encode(all_lyrics)
  inp_embedding = sts_model.encode(input_song)

  sims = cosine_sim(inp_embedding, embeddings)
  
  print("Extracting Best Matches")
  choice_indices = [x for _,x in sorted(zip(sims, options), reverse=True)]
  choice_indices = choice_indices[0:playlist_length]

  for index in choice_indices:
      choices.append(f"{data.artist[index]} - {data.song[index]}")

  print("Playlist Generated:")
  return choices


In [18]:
# Method to get a list of emotion given input text. 
# Emotion of input text is predicted, and 'playlist_length'
# number of songs with that emotion are sampled and returned.
def get_songs_by_emotion(input_text, playlist_length):

  emotion_idx = make_classifier_prediction(input_text)
  emotion = emotions[emotion_idx]
  print(f"Detected Emotion: {emotion}")
  options = emotion_dict[emotion]
  choice_indices = sample(options, playlist_length)
  choices = []
  for index in choice_indices:
    choices.append(f"{data.artist[index]} - {data.song[index]}")
  
  print("Playlist Generated:")
  return choices


In [19]:
get_songs_by_emotion('I want to be enraged', 50)

Detected Emotion: anger
Playlist Generated:


['Kirsty Maccoll - Designer Life',
 'Leo Sayer - Raining In My Heart',
 'Deep Purple - Emmaretta',
 'The Beatles - Jingle Bells',
 "Gloria Estefan - Don't Let The Sun Go Down On Me",
 'Reba Mcentire - Glad I Waited Just For You',
 "Bryan White - I'm Not Supposed To Love You Anymore",
 'Mariah Carey - Get Your Number',
 'Elvis Costello - 15 Petals',
 'Whitesnake - Mean Business',
 'Peter Gabriel - Das Fischernetz',
 'Peter Tosh - Maga Dog',
 'Richard Marx - The Flame Of Love',
 'Meat Loaf - For Crying Out Loud',
 'Lou Reed - Something Happened',
 'Kelly Clarkson - Slamming Doors',
 'Alison Krauss - New Fool',
 "Townes Van Zandt - Honky Tonkin'",
 'Townes Van Zandt - Columbine',
 'Scorpions - Fly To The Rainbow',
 'Phish - Taste',
 'Talking Heads - Pulled Up',
 'Ariana Grande - Rolling In The Deep',
 'Phineas And Ferb - Busted',
 "Grand Funk Railroad - Creepin'",
 'Veruca Salt - Wet Suit',
 'Phish - Boogie On Reggae Woman - Phish',
 'Kim Wilde - Never Trust A Stranger',
 'Alphaville - In

In [ ]:
song_input = input("Enter song lyrics: ")
get_songs_by_text(song_input, 30)

Enter song lyrics: Let me hold you for the last time It's the last chance to feel again But you broke me Now I can't feel anything When I love you, it's so untrue I can't even convince myself When I'm speaking It's the voice of someone else Oh, it tears me up I tried to hold on but it hurts too much I tried to forgive but it's not enough To make it all okay You can't play on broken strings You can't feel anything That your heart don't want to feel I can't tell you something that ain't real Oh, the truth hurts and lies worse How can I give anymore When I love you a little less than before? Oh, what are we doing? We are turning into dust Playing house in the ruins of us Running back through the fire When there's nothing left to save It's like chasing the very last train When it's too late, too late Oh, it tears me up I tried to hold on but it hurts too much I tried to forgive but it's not enough To make it all okay You can't play on broken strings You can't feel anything That your heart 

["Alabama - I Can't Love You Any Less",
 'Westlife - How To Break A Heart',
 'Cher - Hard Enough Getting Over You',
 "Howard Jones - You Can Say It's All Over",
 "Barbra Streisand - Heart Don't Change My Mind",
 "Gloria Estefan - Can't Forget You",
 'Kim Wilde - Falling Out',
 "Donna Summer - Maybe It's Over",
 'Westlife - I Get Weak',
 "Westlife - If Your Heart's Not In It",
 'Cliff Richard - All The Time You Need',
 "Cliff Richard - Love's Salvation",
 'Backstreet Boys - Try',
 'Howard Jones - One Last Try',
 'P!nk - Numb',
 'Air Supply - Tonite',
 'Judds - Cry Myself To Sleep',
 'Johnny Cash - Brand New Dance (featuring June Carter)',
 'Miley Cyrus - FU',
 'Bon Jovi - Heartbreak Eyes',
 'Richard Marx - Slipping Away',
 "Kyla - Doin' Just Fine",
 'Freestyle - Missing You',
 'Yellowcard - Firewater',
 "Conway Twitty - Don't Put Your Hurt In My Heart",
 'Leann Rimes - When You Love Someone Like That',
 'David Guetta - When Love Takes Over',
 "Dusty Springfield - I Can't Give Back The L

# Demo

In [ ]:
PLAYLIST_LENGTH = 30
INPUT_TEXT = "I feel very excited"
get_songs_by_text(INPUT_TEXT, PLAYLIST_LENGTH)